In [1]:
#importing necessary libraries
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
pd.set_option('display.max_columns', None)  # Show all columns in DataFrame output
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('C:\\Users\\Jagdish\\imdb_4.csv')
df.drop(columns = ['canHaveEpisodes'], inplace = True)
df

,startYear,runtimeMinutes,totalCredits,numRegions,ratingCount,castNumber,companiesNumber,writerCredits,directorsCredits,totalNominations,totalMedia,totalReviews,Asia,Africa,Europe,North America,South America,Oceania,Continent Unknown,genre1,genre2,genre3,movie,short,tvEpisode,tvMiniSeries,tvMovie,tvSeries,tvShort,tvSpecial,video,videoGame,rating_bin
0,1894,1,4,7,2092,1,3,0,1,0,11,19,1,0,5,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0,1
1,1892,12,2,6,183,0,0,0,1,0,5,1,1,0,5,0,0,0,0,17855,16787,0,0,1,0,0,0,0,0,0,0,0,1
2,1894,1,1,5,195,0,1,0,1,0,5,0,0,0,4,1,0,0,0,16787,0,0,0,1,0,0,0,0,0,0,0,0,1
3,1894,1,4,6,2238,1,6,0,1,1,9,22,1,0,4,1,0,0,0,16787,16581,0,0,1,0,0,0,0,0,0,0,0,1
4,1896,1,11,21,13115,6,5,0,2,0,33,82,2,0,14,2,1,0,2,16787,16581,0,0,1,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149526,1993,96,11,1,11,0,2,1,1,0,1,0,0,0,0,0,1,0,0,16581,0,0,1,0,0,0,0,0,0,0,0,0,3
149527,2019,14,52,1,15,6,3,1,1,3,10,1,0,0,1,0,0,0,0,51745,16787,0,0,1,0,0,0,0,0,0,0,0,2
149528,2019,29,32,0,12,1,0,0,0,0,5,0,0,0,0,0,0,0,0,5726,0,0,0,0,1,0,0,0,0,0,0,0,1
149529,2011,10,15,0,10,7,0,3,1,0,1,0,0,0,0,0,0,0,0,47408,17855,17763,0,0,1,0,0,0,0,0,0,0,3


# 1% outlier

In [3]:
%%time

# Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Fit Local Outlier Factor (LOF) model
lof = LocalOutlierFactor(n_neighbors=50, contamination=0.01)  # 1% anomalies, no. of neighbors set equivalent to KNN
df['LOF_Score'] = lof.fit_predict(df_scaled)  # -1 = outlier, 1 = inlier

# Extract outliers
df_outliers = df[df['LOF_Score'] == -1]

# Print results
print(f"Total data points: {len(df)}")
print(f"Detected anomalies: {len(df_outliers)}")

Total data points: 149531
Detected anomalies: 1496
CPU times: total: 2min 40s
Wall time: 41.4 s


In [4]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['LOF_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'LOF_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.42
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.27      0.34      2540
           1       0.32      0.19      0.24      3836
           2       0.38      0.39      0.39      7412
           3       0.45      0.67      0.54      9713
           4       0.40      0.27      0.32      5023
           5       0.50      0.16      0.24      1083

    accuracy                           0.42     29607
   macro avg       0.41      0.32      0.34     29607
weighted avg       0.41      0.42      0.40     29607

CPU times: total: 2min 57s
Wall time: 2min 57s


In [6]:
%%time
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.3933866990914311

Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.23      0.30      2540
           1       0.34      0.12      0.18      3836
           2       0.36      0.34      0.35      7412
           3       0.41      0.79      0.54      9713
           4       0.39      0.06      0.11      5023
           5       0.54      0.04      0.07      1083

    accuracy                           0.39     29607
   macro avg       0.41      0.26      0.26     29607
weighted avg       0.39      0.39      0.33     29607

CPU times: total: 578 ms
Wall time: 601 ms


# 5% outlier

In [7]:
%%time

# Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Fit Local Outlier Factor (LOF) model
lof = LocalOutlierFactor(n_neighbors=50, contamination=0.05)  # 5% anomalies, no. of neighbors set equivalent to KNN
df['LOF_Score'] = lof.fit_predict(df_scaled)  # -1 = outlier, 1 = inlier

# Extract outliers
df_outliers = df[df['LOF_Score'] == -1]

# Print results
print(f"Total data points: {len(df)}")
print(f"Detected anomalies: {len(df_outliers)}")

Total data points: 149531
Detected anomalies: 7477
CPU times: total: 2min 46s
Wall time: 42.7 s


In [8]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['LOF_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'LOF_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.42
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.27      0.32      2320
           1       0.33      0.18      0.23      3716
           2       0.38      0.39      0.39      7048
           3       0.46      0.68      0.55      9432
           4       0.42      0.28      0.33      4925
           5       0.48      0.17      0.25       970

    accuracy                           0.42     28411
   macro avg       0.41      0.33      0.35     28411
weighted avg       0.41      0.42      0.40     28411

CPU times: total: 2min 44s
Wall time: 2min 44s


In [10]:
%%time
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.39354475379254517

Classification Report:
               precision    recall  f1-score   support

           0       0.32      0.25      0.28      2320
           1       0.35      0.13      0.19      3716
           2       0.35      0.35      0.35      7048
           3       0.42      0.75      0.54      9432
           4       0.37      0.10      0.16      4925
           5       0.46      0.05      0.10       970

    accuracy                           0.39     28411
   macro avg       0.38      0.27      0.27     28411
weighted avg       0.38      0.39      0.35     28411

CPU times: total: 453 ms
Wall time: 446 ms


# 10% outlier

In [11]:
%%time

# Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Fit Local Outlier Factor (LOF) model
lof = LocalOutlierFactor(n_neighbors=50, contamination=0.1)  # 10% anomalies, no. of neighbors set equivalent to KNN
df['LOF_Score'] = lof.fit_predict(df_scaled)  # -1 = outlier, 1 = inlier

# Extract outliers
df_outliers = df[df['LOF_Score'] == -1]

# Print results
print(f"Total data points: {len(df)}")
print(f"Detected anomalies: {len(df_outliers)}")

Total data points: 149531
Detected anomalies: 14953
CPU times: total: 2min 40s
Wall time: 41.1 s


In [12]:
# Remove anomalies (LOF_Score == -1) to get cleaned data
df_clean = df[df['LOF_Score'] == 1]

# Define features (X) and target (y)
X = df_clean.drop(columns=['rating_bin', 'LOF_Score'])  # Exclude target and LOF_Score columns
y = df_clean['rating_bin']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
%%time
# Initialize and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=50, metric='manhattan', weights = 'distance')  # You can adjust the number of neighbors
knn.fit(X_train, y_train)

# Make predictions
y_pred = knn.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.43
Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.29      0.35      2226
           1       0.34      0.20      0.25      3520
           2       0.40      0.40      0.40      6843
           3       0.46      0.67      0.54      8960
           4       0.41      0.29      0.34      4492
           5       0.51      0.17      0.26       875

    accuracy                           0.43     26916
   macro avg       0.43      0.34      0.36     26916
weighted avg       0.42      0.43      0.41     26916

CPU times: total: 2min 18s
Wall time: 2min 18s


In [14]:
dtc = DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter = 'random')# Initialize the Decision Tree Classifier
dtc.fit(X_train, y_train)

# Make predictions
y_pred = dtc.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.40403477485510475

Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.24      0.30      2226
           1       0.38      0.14      0.20      3520
           2       0.37      0.37      0.37      6843
           3       0.42      0.77      0.54      8960
           4       0.39      0.10      0.15      4492
           5       0.41      0.05      0.08       875

    accuracy                           0.40     26916
   macro avg       0.40      0.28      0.28     26916
weighted avg       0.40      0.40      0.35     26916

